# Maximum Gift Exchange Chaos

A model designed to explore the impact of different gift exchange rules (e.g. White Elephant, Green Grinch, etc.). The primary goal is to find the rules that lead to maximum chaos, as defined as maximum number of gift steals (possibly -- probably? -- within some time constraint).

## Gift Exchange Model

We can model a gift exchange as a game with an equal number of `G` `gifts` and `P` `players`. Players `value` each gift according to their personal preferences.

    
### Rule Variants
Some gift exchange variants use slightly different rules. These variants can be mixed and matched.

#### Additional end-game swapping
This variant adds extra swaps at the end of the game after `p1` makes their swap. If `p1` does swap with another player's gift, that player becomes active and repeats step 4, and so on until the active player elects not to swap or there are no more eligible gifts.

#### All gifts start unwrapped
In this variant, all gifts in the pool are unwrapped. This means that, instead of players learning their preferences as gifts are opened, all players have complete information about their gift preferences throughout the entire game.

#### Per-round stealing constraints
- A gift can only be stolen `St` times per round, regardless of who is doing the stealing. In other words, once a gift is stolen within a given round, it is ineligble for the remainder of that round.
- No more than `Sg` gifts can be stolen per round. Once `Sg` is reached, the round concludes and a new round begins by the next player becoming active in step 2.

#### Per-player stealing constraints
In this variant, there is also a cap on the number of times `Sp` each *player* can be stolen from. For example, once a given player `Sp_n` has been stolen from `Sp_n = Sp` times, the gift they hold at that point is now ineligible. 

#### No gift stealing constraint
In this variant, there is no cap on the number of times a given gift can be stolen. In other words, `S = infinity`.

### Gift Values
Players have personal preferences and value gifts accordingly. Each `player` has a (potentially) unique subjective `subj_value` for each `gift`. We can therefore model gift subjective value as a `G` by `P` matrix of values `subj_value_gp in range(100)`.

But, if we allow for the notion that some gifts are objectively better than others (almost all gifts will be valued higher than a spray-painted tamborine, for example), we can model `subj_value` as normally distributed variance around a common `obj_value` level. 

In [12]:
import numpy as np
import math
import pandas as pd
import scipy.stats as stats

## Inititial Conditions, Simulation Parameters, and Setup

### First, we set some initial conditions for our model

In [2]:
# game setup
player_count = 10
gift_count = player_count

# gift values setup
gift_value_scale = (0,1)
gift_obj_value_distr = {'mean': .5, 'stdev': .2}
gift_subj_value_stdev = .2

### Next, we create two classes to model the simulation's state

#### Gift Class

Each **gift** has the following attributes:
- `is_unwrapped`: a boolean marking whether the gift is open (`True`) or wrapped (`False`)
- `owned_by`: the id of the `Player` that currently owns the gift; otherwise `None`
- `brought_by`: the id of the `Player` that brought the gift to the game
- `steals`: a counter for the number of times the gift is stolen (including swaps)
- `obj_value`: the gift's objective value, set randomly within the `gift_value_scale` based on the `gift_obj_value_distr` parameter
- `is_eligible`: a boolean marking whether the gift can still be stolen; must be `False` if either `owned == False` or `steals < gift_steals_cap`

In [252]:
class Gift:    
    def __init__(self, id, value_scale, value_distr, wrapped):
        self.id = id
        self.is_unwrapped = not wrapped
        self.owned_by = None
        self.steals = 0
        self.is_eligible = True
        self.obj_value = Gift.set_obj_value(value_scale, value_distr)
        self.brought_by = None
    
    # helper function to create a truncated normal distribution
    @staticmethod
    def create_trunc_norm_distr(scale, mean, stdev):
        lower, upper = scale[0], scale[1]
        return stats.truncnorm(
            (lower - mean) / stdev,
            (upper - mean) / stdev,
            loc = mean,
            scale = stdev)
    
    # set an objective value for a gift
    @staticmethod
    def set_obj_value(scale, distr):
        mean, stdev = distr['mean'], distr['stdev']
        trunc_norm_distr = Gift.create_trunc_norm_distr(scale, mean, stdev)
        value_array = trunc_norm_distr.rvs(1)
        return value_array[0]
    
    # TODO: property to increment self.steals and check if self.is_eligible should be changed
    
    @staticmethod
    def available_gifts(gifts):
        available_gifts = []
    
        for g in range(gift_count):
            gift = gifts[g]
            if gift.is_eligible and (gift.owned_by is None):
                available_gifts.append(gift)
                
            elif not gift.is_eligible:
                pass
                
            elif players[gift.owned_by].is_stealable:
                available_gifts.append(gift)
                
        return available_gifts
    


In [300]:
# generate gifts 

gifts = []

for g in range(gift_count):
    gifts.append(Gift(g, gift_value_scale, gift_obj_value_distr, wrapped = True))

In [301]:
[g.obj_value for g in gifts]

[0.4461279330133322,
 0.2993714348870825,
 0.8892112155218264,
 0.7013878328086486,
 0.5338105621577487,
 0.652302592384453,
 0.4764548775980537,
 0.40667896245798707,
 0.6720237289428854,
 0.2565087577635378]

#### Player Class

Each **player** has the following attributes:
- `brought_gift`: the id of the `Gift` the player brought to the game; the player always knows their subjective value of this gift
- `owns_gift`: the id of the `Gift` the player currently owns; otherwise `None`
- `steals`: a counter for the number of times the player has a gift *stolen from them*
- `is_stealable`: a boolean marking whether the player can still be stolen *from*
- `is_active`: a boolean marking whether it's this player's turn to select, steal, or swap a gift
- `subj_values`: Subjective value for a given player&rightarrow;gift is set randomly around the gift's `obj_value` based on the `gift_subj_value_stdev` parameter

In [293]:
class Player:
    def __init__(self, id, value_scale, subj_value_stdev, strategy, giftIDs):
        self.id = id
        self.brought_gift = id
        self.strategy = strategy
        self.steals = 0
        self.is_stealable = True
        self.owns_gift = None
        self.is_active = True if id == 0 else False # only the first player starts out as active
        self.subj_values = {}
        
        gifts[p].brought_by = p
        
        for gift in giftIDs:
            self.subj_values[gift] = self.set_gift_subj_value(gift_value_scale, 
                                                              gifts[gift].obj_value, 
                                                              gift_subj_value_stdev)
    
    # set a subjective value for a gift around its objective value
    # @private --can I do this?
    def set_gift_subj_value(self, scale, obj_value, stdev):
        distr = Gift.create_trunc_norm_distr(scale, obj_value, stdev)
        value_array = distr.rvs(1)
        return value_array[0]
    
    #TODO: a function that executes the player's strategy
    def make_decision(gifts):
        strategy = self.strategy
        can_steal = []
        pass
    
    def get_subj_value_of(self, gifts):
        return { key:value for key, value in self.subj_values.items() if key in gifts }
    
    # get unwrapped gifts that self knows the value for, i.e. are either unowned or stealable or brought by self
    def available_known_gifts(self, gifts):
        available_known_gifts = set(Gift.available_gifts(gifts))
        available_known_gifts.add(gifts[self.brought_gift])
        return list(available_known_gifts)
    
    # get dictionary of available known gifts and their subjective values for self
    def sorted_available_known_gift_values(self, gifts):
        available_known_gifts = self.available_known_gifts(gifts)
        print(available_known_gifts)
        unsorted = self.get_subj_value_of(available_known_gifts)
        print(unsorted)
        return dict(sorted((key,value) for (key, value) in unsorted.items()))
    
    # get self's best available known gift
    def best_available_known_gift(self, gifts):
        best_gift = list(self.sorted_available_known_gift_values(gifts)[0].items())[0] 
        return best_gift
    
# generate players

players = []
for p in range(player_count):
    players.append(Player(p, gift_value_scale, 
                          gift_subj_value_stdev, 
                          'steal_gift_above_threshold', 
                          range(len(gifts))))

In [295]:
#players[9].sorted_available_known_gift_values(gifts)
9 in gifts

False

In [177]:
[p.subj_values for p in players]

[{0: 0.3144859203431629,
  1: 0.23818354013146623,
  2: 0.6141656318286548,
  3: 0.11285935705429642,
  4: 0.6723935810766533,
  5: 0.8027452822603545,
  6: 0.9679609189821892,
  7: 0.3262007303519398,
  8: 0.18959919956636032,
  9: 0.31291890030863684},
 {0: 0.4787236474076305,
  1: 0.5340702829131274,
  2: 0.36962323207938136,
  3: 0.12193022008492518,
  4: 0.589963410605801,
  5: 0.4554333879541182,
  6: 0.7558308648991392,
  7: 0.6345853727588086,
  8: 0.12635739483040537,
  9: 0.28392014185881015},
 {0: 0.9444454568753493,
  1: 0.6551575321491057,
  2: 0.5454148091937284,
  3: 0.4246996715288568,
  4: 0.6178092291197952,
  5: 0.8305802118926727,
  6: 0.9187441909042843,
  7: 0.9099014944358509,
  8: 0.373890336067254,
  9: 0.5909654645632176},
 {0: 0.4459790636106349,
  1: 0.8464554080169295,
  2: 0.29084570179979324,
  3: 0.27605518599486406,
  4: 0.9564007042252247,
  5: 0.6960603112575642,
  6: 0.959510069511526,
  7: 0.6220053562029703,
  8: 0.55369527723988,
  9: 0.7062667548

## Policies

### Player Strategies

1. Steal second most valuable unwrapped gift
2. Steal most valuable unwrapped gift
3. Steal open gift if `subj_value` is greater than...
    1. Average of `subj_values` of the unwrapped gifts
    2. The maximum `subj_values` of the unwrapped gifts
    3. An arbitrary threshold, e.g. 80
4. Steal an open gift if about to become unstealable

Possible strategies (from https://github.com/BenCasselman/YankeeSwap):
1. Player steals with probability p = (number of gifts taken) / N (naive)
2. Player always steals most valuable gift available
3. Player always steals second-most-valuable gift available (if only one gift is available, player steals that one)
4. Player never steals
5. Player steals if any stealable gift has value (to them) greater than estimated underlying value of average gift
6. Player steals about-to-be unstealable gift if one available greater than estimated underlying value of avg gift
7. Same as #5 but factor in knowledge of gift player brought.
8. Ghosh-Mahdian: Player steals if best available gift has value > theta


To keep things simple at first, we'll use a single strategy for all players: 
- steal the highest `subj_value` open gift if `subj_value` >= 0.67

In [ ]:
# player strategies

# in each of the following, "best" gift refers to the gift with the highest subjective value
strategies = [
    'steal_gift_above_threshold', # if the best unwrapped gift is owned / eligible, steal it if above steal_threshold
    'best_unwrapped_gift', # steal the best unwrapped gift if its owned / eligible, otherwise select it from pool
    'steal_second_best_gift', # if the best unwrapped gift is owned, steal the second best owned / eligible gift
    'steal_almost_ineligible_gift', # if stealing a gift would turn it ineligible, steal it if above steal_threshold
]

steal_threshold = gift_value_scale[1] * 2/3

# helper functions
    
def get_unowned_gifts():
    # find all unowned giftsIDs
    pass

# TODO: convert this function into a cadcad policy function
def strategy_steal_gift_above_threshold(player, steal_threshold):
    best_available_known_gift = player.best_available_known_gift(gift_ids)
    
    best_available_known_giftID = best_available_known_gift[0]
    best_available_known_gift_value = best_available_known_gift[1]
    
    if 
    
    
    # eligible_known_gift_values = sorted_subj_gift_values(get_eligible_known_gifts_for(player))
    
    # best_eligible_known_gift = eligible_known_gift_values[0][0]
    # best_eligible_known_gift_value = eligible_known_gift_values[0][1]
    
    # if best_eligible_known_gift is in get_unowned_gifts():
        # gift_intention = (open_gift, best_eligible_known_gift)
        
    # elif best_eligible_known_gift_value >= steal_threshold:
        # gift_intention = (steal_gift, best_eligible_known_gift)
        
    # else: #i.e. if best_eligible_known_gift is owned and under steal_threshold
        # random_gift = select an unowned gift at random
        # gift_intention = (open_gift, random_gift)
    
    pass


### Game Rules and Logic

#### Basic Rules
Each player brings one gift to the game and leaves with a (typically) different gift.

The game begins with all gifts wrapped, i.e. each player knows the content of only the gift they brought and none other.

A player order is determined (by some arbitrary method, e.g. random).

1. The first player `p1` selects one of the gifts from the set of unwrapped gifts and opens it.
2. The next player `p2` is now active, and has the option to either:
    1. Open a gift from the pool, leading to a repeat of step 2 with `p3` active
    2. Steal an eligible open gift, leading to a repeat of step 2 with `p1` active
3. Repeat step 2 until all gifts have been opened (i.e. there are no gifts remaining in the pool).
4. `p1` is now active again and has the option to (forcibly) swap with another player for their eligible gift. The game ends once `p1` has swapped or decided to not swap.

A new **round** begins when a new gift is opened from the pool.

##### Stealing
- Each gift can only be stolen up to `gift_steals_cap` times, including swaps. A given gift `G_n` is eligible for stealing (or swapping) while `gift_steals_n < gift_steals_cap`. For example, when `gift_steals_cap = 3`, each gift can only be stolen 3 times. After that, the gift is no longer eligible and no player may steal it or swap for it; a player holding an ineligible gift will leave the game with that gift.
- Within a given round, gifts cannot be stolen back by the player who began the turn holding that gift.

    1. If they do not swap, the game ends.
    2. If they do swap with another player, step 4 repeats with that swapped player active

In [154]:
# game rules, used as simulation parameters
rule_params = {
    'gift_steals_cap': 3, # if none, there is no cap
    'player_steals_cap':  None, # if none, there is no cap
    'round_total_steals_cap': None, # number of total steals allowed per round
    'round_gift_steals_cap': None, # number of steals allowed per round per gift
    'single_swap': True, # True: only the first player is allowed to swap gifts at the end of the game; True: unlimited swaps are allowed
    'gifts_start_wrapped': True
}
def action_swap_gifts(player):
    # swap owned gift for gift with highest subjective value
    
    # owned_gift = 
    
    # other_gifts = 
    
    # best_other_gift = sorted_subj_gift_values(player, other_gifts)[0][1]
    
    # gift_intention = (swap_gifts, best_other_gift)
    
    pass

## State Update Functions

In [146]:
# player actions

def open_gift(active_player, gift): 
    # if gift is wrapped:
    
        # change gift's open attribute to True

        # change active_player --> gift owns edge to True
        
        # change active_player active attribute to False
        
        # change players.index(active_player + 1) active attribute to True
        
    # else:
        # throw error
    
    pass


def steal_gift(active_player, gift):
    # if gift is open and stealable and owner player is stealable:
        
        # increment owner player steals attribute
        
        # if owner player steals attribute >= player_steals_cap:
            # set owner player stealable attribute to False
            
        # change previous owner player --> gift owns edge to False
        
        # change active_player --> gift owns edge to True
        
        # increment gift steals attribute
        
        # if gift steals attribute >= gift_steals_cap:
            # set gift stealable attribute to False
            
        # change active_player active attribute to False
            
        # change previous owner player active attribue to True
            
    # else:
        # throw error
        
    pass

def swap_gifts(active_player, target_gift, single_swap):
    # if gift is open and stealable and owner player is stealable:
    
        # increment owner player steals attribute
        
        # if owner player steals attribuet >= player_steals_cap:
            # set owner player stealable attribute to False
            
        # change previous owner player --> target_gift owns edge to False
        
        # change active_player --> target_gift owns edge to True
        
        # active_player_gift = active_player's current gift
        
        # change previous owner player --> active_player_gift owns edge to True
        
        # change active_owner --> active_player_gift owns edge to False
        
        # increment gift steals attribute
        
        # if gift steals attribute >= gift_steals_cap:
            # set gift stealable attribute to False
            
        # change active_player active attribute to False
            
        # if single_swap = False:
            # change previous owner player active attribute to True
            
    # else:
        # throw error
            
    pass

def play_round():
    active_player = pass
    
    pass


## Partial State Update Blocks

In [ ]:
partial_state_update_blocks = [
    { # initiate conditions
        'policies': {
            
        },
        'variables': {
            'players': , # initiate players
            'gifts':  # initiate gifts
        }
    },
    { # complete a single round
        'policies': {
            
        },
        'variables': {
            'players': ,
            'gifts': 
        }
    }
]

## Simulation Execution

In [ ]:
from cadCAD.configuration.utils import config_sim
simulation_parameters = config_sim({
    'T': player_count,
    'N': 1,
    'M': rule_params
})

# The configurations above are then packaged into a `Configuration` object
config = Configuration(initial_state = initial_conditions, #dict containing variable names and initial values
                       partial_state_update_blocks = partial_state_update_blocks, #dict containing state update functions
                       sim_config = simulation_parameters #dict containing simulation parameters
                      )

# Run the simulations
exec_mode = ExecutionMode()
exec_context = ExecutionContext(exec_mode.single_proc)
executor = Executor(exec_context, [config]) # Pass the configuration object inside an array
raw_result, tensor = executor.execute() # The `execute()` method returns a tuple; its first elements contains the raw results

## Simulation Results

In [ ]:
df = pd.DataFrame(raw_result)